<a href="https://colab.research.google.com/github/arthurgamaruiz/Projeto-Titanic/blob/main/An%C3%A1liseTitanicParte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizando a base de treino

In [ ]:
# Importação dos módulos

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

In [ ]:
# Importação dos dados

treino = pd.read_csv("/content/drive/MyDrive/Projeto Titanic/titanic/train.csv")
treino.head()

In [ ]:
# Gera relatório em html
profile = ProfileReport(treino, title="titanic_treino")
profile.to_file("titanic_treino.html")

In [ ]:
# Verificando informações da base
treino.info()

In [ ]:
# Verificando o tipo dos dados
treino.dtypes.value_counts()

In [ ]:
# Valores nulos
treino.isnull().sum().sort_values(ascending=False).head()

# Visualizando a base de teste

In [ ]:
# Importação dos dados

teste = pd.read_csv("/content/drive/MyDrive/Projeto Titanic/titanic/test.csv")
teste.head()

In [ ]:
# Verificando informações da base
teste.info()

In [ ]:
# Verificando o tipo dos dados
treino.dtypes.value_counts()

In [ ]:
# Valores nulos
teste.isnull().sum().sort_values(ascending=False).head()

# Tratamento dos dados

In [ ]:
# Shape da base de treino
treino.shape

In [ ]:
# Verificando a cardinalidade dos dados
"""
colunas com cardinalidade alta não devem ser utilizadas na modelagem,
pois podem tornar o modelo mais suscetível a erros de generalização.
Eliminar tais colunas
"""
treino.nunique().sort_values(ascending=False)

In [ ]:
# Eliminando colunas da base de treino
treino = treino.drop(['Name', 'Ticket', 'Cabin'], axis=1)    # axis=1 --> coluna

In [ ]:
# Eliminando colunas da base de teste
teste = teste.drop(['Name', 'Ticket', 'Cabin'], axis=1)

A coluna 'Age' possui valores nulos, então deve ser tratada (substituiremos pela média)

In [ ]:
# Média das idades
treino.Age.mean()  #ou treino['Age'].mean()

In [ ]:
# Localizar colunas com valores nulos e substituir pela média
treino.loc[treino.Age.isnull(), 'Age'] = treino.Age.mean()

In [ ]:
# Repetindo para a base de teste
teste.loc[teste.Age.isnull(), 'Age'] = teste.Age.mean()

Para a base de treino, temos dois valores vazios na coluna **Embarked**. Podemos verificar todos os valores nessa coluna e substituir os valores vazios pela **moda**

In [ ]:
# Verificando valores da coluna Embarked
treino.Embarked.value_counts()

In [ ]:
# Selecionando a moda dessa coluna
treino.Embarked.mode()[0]

In [ ]:
# Substituindo os valores vazios
treino.loc[treino.Embarked.isnull(), 'Embarked'] = treino.Embarked.mode()[0]

# Base de teste não possui valores nulos na coluna Embarked

Para finalizar, para a base de teste há um valor vazio na coluna Fare. Vamos substituir pela média

In [ ]:
teste.loc[teste.Fare.isnull(), 'Fare'] = teste.Fare.mean()

Verificando novamente os valores vazios

In [ ]:
# Base de treino
treino.isnull().sum().sort_values(ascending=False).head()

In [ ]:
# Base de teste
teste.isnull().sum().sort_values(ascending=False).head()

Considerando apenas colunas que não são texto

In [ ]:
# Colunas de texto selecionadas
col_treino_nr = treino.columns[treino.dtypes != 'object']
col_treino_nr

In [ ]:
# Selecionando apenas os valores numéricos da base de treino
treino_nr = treino.loc[:, col_treino_nr]
treino_nr

In [ ]:
# Para a base de teste
col_teste_nr = teste.columns[teste.dtypes != 'object']
col_teste_nr

In [ ]:
# Valores numéricos da base de teste
teste_nr = teste.loc[:,col_teste_nr]
teste_nr

# Selecionar algum modelo para classificar os dados:
- Árvore de classificação: https://scikit-learn.org/stable/modules/tree.html;
- Classificação dos vizinhos mais próximos: https://scikit-learn.org/stable/modules/neighbors.html;
- Regressão Logística: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html.

Antes de utilizar os algoritmos, devemos separar a base de treino em treino e validação

In [ ]:
# Importanto train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
# Separando a base de treino em X e y
X = treino_nr.drop(['PassengerId', "Survived"], axis=1)
y = treino.Survived

In [ ]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

Árvore de decisão

In [ ]:
# Fazendo a importação
from sklearn import tree

In [ ]:
# Criando o Classifier
clf_ac = tree.DecisionTreeClassifier(random_state=42)

In [ ]:
# Fazendo o ajuste com os dados
clf_ac.fit(X_train, y_train)

In [ ]:
# Fazendo a previsão
y_pred_ac = clf_ac.predict(X_val)

KNeighboorsClassifier

In [ ]:
# Importação
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Criando o classificador
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Fazendo o ajuste com os dados
knn.fit(X_train, y_train)

In [ ]:
# Fazendo a previsão
y_pred_knn = knn.predict(X_val)

Regressão logística

In [ ]:
# Importando
from sklearn.linear_model import LogisticRegression

In [ ]:
# Criando classificador
clf_rl = LogisticRegression(random_state=42)

In [ ]:
# Fazendo o ajuste com os dados
clf_rl.fit(X_train, y_train)

In [ ]:
# Fazendo a previsão
y_pred_rl = clf_rl.predict(X_val)


# Avaliar os modelos

Para realizar essa análise, podemos utilizar:
- Acurácia:https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html;
- Matriz de confusão: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html.

Avaliando acurácia

In [ ]:
# Importando
from sklearn.metrics import accuracy_score

In [ ]:
# Para a árvore
accuracy_score(y_val, y_pred_ac)

In [ ]:
# Para o knn
accuracy_score(y_val, y_pred_knn)

In [ ]:
# Para a regressão logística
accuracy_score(y_val, y_pred_rl)

Avaliando a matriz de confusão

In [ ]:
# Importando
from sklearn.metrics import confusion_matrix

In [ ]:
# Para a árvore
confusion_matrix(y_val, y_pred_ac)

In [ ]:
# Para o knn
confusion_matrix(y_val, y_pred_knn)

In [ ]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

Fazendo a previsão para os dados de teste com o melhor modelo (regressão logística)

In [ ]:
# Visualizando X_train
X_train.head()

In [ ]:
# Visualizando a base de teste (apenas com valores numéricos)
teste_nr.head()

In [ ]:
# Eliminando a coluna de id para a base de teste ser igual à de treino
X_teste = teste_nr.drop(['PassengerId'], axis=1)

In [ ]:
# Utilizando regressão logística na base de teste
y_pred = clf_rl.predict(X_teste)

In [ ]:
# Criando uma coluna comm precisão na base de teste
teste['Survived'] = y_pred

In [ ]:
# Selecionando apenas a coluna de Id e Survived para enviar
envio = teste[['PassengerId', 'Survived']]
envio

In [ ]:
# Exportando para um arquivo csv
envio.to_csv('envio.csv', index=False)    #index=False retira a numeração das linhas